In [1]:
import pandas as pd
import numpy as np
import os
import requests
from io import BytesIO
import json
import yadisk
from datetime import datetime, date, timedelta
import re
import gspread
from oauth2client.service_account import ServiceAccountCredentials

import config
from yandex_disk_func import *
from parse_functions import *
from google_connector import *

# # указываем путь и название файла с токеном для Яндекс Диск
file_path = r'C:\Users\o.bogomolov\Desktop\Jupyter_notebook\35_mediaplan_parser'
# забираем токен для подключения к гугл
service_key = config.service
google_sheet_link = 'https://docs.google.com/spreadsheets/d/11hF_txYEbKrQImRQ0x3Fia3PyJvDAqtpGI25idV7oa4/edit?usp=sharing'
gmail = config.gmail
sheet_name = 'prog'

# обычная ссылка на доступ к папке одного данного ФЛАЙТА из личного кабинета
public_key = config.public_key 

# забираем список папок в нужной нам директории
res = get_yandex_disk_folders(public_key)
yandex_folders = res.json() # парсим ответ

200


In [2]:
# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)
# Задаем ширину столбцов по контенту
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.options.mode.chained_assignment = None
# pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [3]:
# создаем словарь, где сохраним каждый продукт в отдельный датаФрейм со своим ключом
main_dict = {}

In [4]:
get_data_from_ya_folder(yandex_folders, main_dict, flag='prog')                     


/01_yandex
/02_prog
beeline_plan_игроник_ингосстрах_кис_лето_banners_020724
firstdata_игроник_ингосстрах_срм_media_28.08.2024
    mediaplan без ск
firstdata_игроник_национальная лотерея_принцесса нури_срм_media_27.08.2024 (1)
    mediaplan 1000к
    mediaplan 800к
    mediaplan 600к
hybrid_бронь_игроник_национальная лотерея_день рождения нл_klientskiy_plan_20240920 (1)
    медиаплан
mp_mobidriven_игроник_ингосстрах_ хореографы_24.09 (1)
    мп_4 дня
    мп_4 неделе
mp_mobidriven_игроник_ингосстрах_инго экосистема_05.09
    мп
mp_mobidriven_игроник_национальная лотерия_27.08 (1)
    мп
mp_roxot_игроник_psb_brand-booster_sep-dec24
    псб bb banner
    псб bb olv
plan_игроник_рсхб_beeline_свой вклад_banners_160724 (1)
plan_ингосстрах_ beeline_хореографы_banners_240924 (3)
beeline_plan_igronik_бриф по тендерам_101024
игроник_ segmento __ рсхб __ olv_2024-07-03
игроник_дом рф_smart tv_hybrid_klientskiy_plan_20240725
    медиаплан (ск 40%)
    медиаплан (ск 50%)
игроник_псб имидж спецпроект

In [6]:
df = pd.concat(main_dict, ignore_index=True)
df = df.fillna('')

df['views'] = df['views'].astype('float').round(0)
int_columns = ['reach', 'impressions', 'clicks', 'views']
df[int_columns] = df[int_columns].apply(lambda x: x.astype('int'))

In [7]:
float_columns = ['unit price', 'budget_without_nds', 'budget_nds', 'vtr, %']
df[float_columns] = df[float_columns].apply(lambda x: x.astype('float')).round(2)

In [ ]:
df

In [ ]:
test = df[df['report_name']=='рсхб_игроник_ segmento _ рсхб_up_2024']

In [8]:
test = df[df['sheet_name']=='mediaplan 1000к']

In [9]:
test

,supplier,report_name,sheet_name,brand,period,source,site/ssp,placement,targeting,geo,soc_dem,ad copy format,rotation type,unit price,frequency,reach,impressions,clicks,budget_without_nds,budget_nds,views,"vtr, %"
4,firstdata,firstdata_игроник_национальная лотерея_принцесса нури_срм_media_27.08.2024 (1),mediaplan 1000к,Национальная лотерея // Принцесса Нури,6 months,Платформа First Data,,,"ГЕО: РФ\nЦА: Ж, 35-45. Доход В, ВС. Покупатели чая и чайной продукции: Принцесса Нури, Азерчай, Ява, Riston, Майский, Ахмад, Dilmah, Ява, Милфорд, Липтон и др. Покупатели кондитерских изделий: безе, варенье, джем, повидло, грильяж, желе, зефир, пастила, конфеты, ирис, халва, помадка, муссы, шоколад, торты и др\nИсточники: ОФД",рф,"ж, 35-45. доход в, вс.",Баннеры / Универсальные Баннеры,CPM,240.0,12,2083333,25000000,50000,6000000.0,7200000.0,0,0.0


In [ ]:
test.iloc[1]

In [ ]:
test[['reach', 'impressions', 'clicks', 'budget_without_nds']].sum()

In [ ]:
df

In [ ]:
# создаем подключение к Гуглу
client = create_connection(service_key)

In [ ]:
# прописываем путь и подключаемсяк нужному листу Гугл докса
sh = client.open_by_url(google_sheet_link)
sh.share(gmail, perm_type='user', role='writer')
google_sheet = sh.worksheet(sheet_name)

In [ ]:
# выгружем наш датаФрейм на лист
google_sheet.clear()
export_dataframe_to_google_sheet(google_sheet, final_report)

In [ ]:
main_dict['beeline_plan_игроник_ингосстрах_кис_лето_banners_020724']

In [ ]:
main_dict['plan_игроник_рсхб_beeline_свой вклад_banners_160724 (1)'].iloc[0]

In [ ]:
main_dict['plan_ингосстрах_ beeline_хореографы_banners_240924 (3)']

In [ ]:
main_dict['beeline_plan_igronik_бриф по тендерам_101024']